In [1]:
import numpy as np
import scipy as sp

In [2]:
from quadrotor.dynamics import QuadrotorDynamicsBase, QuadrotorState
from quadrotor.controller import ControllerBase, QuadrotorCommands
from quadrotor.trajectory import TrajectoryBase, TrajectoryState

### Before diving in
Before diving in, I recommend going through the text part in the first notebook and the base files in the repository!

In [3]:
from sym import Rot3

def dRot3(R: Rot3, omega: np.ndarray) -> Rot3:
    """
    NOTE(roy): If interested, we can discuss the math here later. There are other
    ways of doing this but to keep the ode_int formulation, we specifically require
    dq/dt. Refer to 'Baseile Graf - Quaternions and Dynamics' or (for intuition)
    https://math.stackexchange.com/questions/1896379/how-to-use-the-quaternion-derivative
    """
    quat = np.array(R.to_storage())  # Convert to raw quaternion
    (q0, q1, q2, q3) = quat  # xyzw storage, see https://symforce.org/api-gen-cpp/class/classsym_1_1Rot3.html

    G = np.array([[ q3,  q2, -q1, -q0],
                [-q2,  q3,  q0, -q1],
                [ q1, -q0,  q3, -q2]])
    quat_dot = (G.T @ omega) / 2
    return Rot3.from_storage(quat_dot)

### Building a 2D planar quadrotor model
In this second notebook, we'll build a planar quadrotor model as discussed in the slides. Note that most of our base classes (e.g. `QuaternionState`, etc...) are built for a 3D quadrotor. This means that we'll effectively just ignore a dimension. In this case, we'll use `x` and `z` as our horizontal/vertical plane and assume the quadrotor is only able to rotate about the `y` axis that goes into the plane. In the `QuadrotorCommands` this means we'll use only the first 2 rotor rates and pretend the back two don't exist!

#### Your work
This is the first 'real' notebook and we've gone through material pretty fast. To get started, I've provided some ground-work for certain things. Throughout the code, you'll see `raise NotImplementedError()` exceptions in places where you're expected to write code (so replace them with your actual implementation). Please refer to the first notebook for more info on the program structure and coding guidelines. Given that we're doing a lot of math implementation here, the code is fairly dense, and I want you to err on the side of overclarification.

You'll also find some `NOTE(roy)` comments in there sometimes, those are comments that I think are valuable for you to read, or provide extra sources if you're interested.

#### Model parameters
Let's first define a set of parameters to work with. We'll use a `dataclass` structure ([this](https://realpython.com/python-data-classes/) is a decent intro but the gist of it is that we don't need to define an `__init__` or `__repr__` function and can just add our fields).

In this case, I'm using parameters for a [Skydio X2](https://www.skydio.com/skydio-x2) drone. Later during the course we'll play with other parameters to see how things change (e.g. a [Skydio S2](https://www.skydio.com/skydio-2-plus-enterprise) or a [CrazyFly](https://www.bitcraze.io/products/crazyflie-2-1/)). From a control point of view, this drone is a little strange, it's relatively heavy and uses three-blade propellors, which means thrust is higher at lower rotors speeds.

In [4]:
from dataclasses import dataclass

@dataclass
class Params:
    """A little utility class to hold our quadrotor parameters"""
    mass: float = 1.352  # [kg]
    inertia: float = 9.8e-2  # [kg m2]

    rotor_diameter: float = 10 * 0.0254  # [m] 10 inches

    static_thrust_coefficient: float = 0.14553  # [-] Thrust coefficient
    static_torque_coefficient: float = 0.01047  # [-] Torque coefficient

    arm_length: float = 0.3814 / 2.0  # [m] Arm length

    # Universal constants
    g: float = 9.80665  # [m/s2] Gravity
    rho: float = 1.225  # [kg/m3] Sea-level density

    def rotor_model(self, static_coefficient: float) -> float:
        # NOTE(roy) I'm just including this for completeness sake, this means we can e.g. see how air density
        # influences our calculations. Refer to this source for derivation:
        # https://web.mit.edu/16.unified/www/FALL/thermodynamics/notes/node86.html
        return self.rho * static_coefficient * (self.rotor_diameter ** 4) / (4 * np.pi ** 2)
    
    @property
    def k_thrust(self):
        return self.rotor_model(self.static_thrust_coefficient)

    @property
    def k_torque(self):
        return self.rotor_model(self.static_torque_coefficient)

# Simply treat as a global
p = Params()

#### Creating a dynamics model
This was also mentioned in the previous notebook, but we'll make use of classes and inheritance here. If this is unclear to you, [this](https://ioflood.com/blog/python-inheritance/) might be a good source.

**If there is any use to me going over Python concepts, please let me know. I'm happy to discuss in a 1:1 or if there's enough interest I'll devote part of a session to this!**


#### Using scipy's ODE solver
As discussed in the project session, we'll use a built-in ODE solver (it's a fun/interesting exercise to do this yourself, i.e. try implementing forward Euler!). We'll use scipy's `solve_ivp` (solve initial value problem) to approach this. I left a comment in the code where you're expected to use this, [refer to the documentation here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html). Note that for each timestep, we're defining a new ODE, so for each timestep, make sure to integrate from 0 to dt.

In [5]:
class PlanarQuadrotorDynamics(QuadrotorDynamicsBase):
    def __init__(self) -> None:
        # NOTE(roy): I advise you to scroll through dynamics.py and see what the __init__
        # function in the base class does!
        super().__init__()

    @staticmethod
    def rotor_thrust_model(rotor_rates: np.ndarray) -> np.ndarray:
        # NOTE(roy): Implement the simple rotor trust model we discussed in the slides
        # (For fun) to see how complex this can get, see for example: https://arxiv.org/pdf/1601.00733.pdf
        return p.k_thrust * rotor_rates ** 2

    def step(self, t: float, input: QuadrotorCommands) -> QuadrotorState:
        # NOTE(roy) Use only the first two rotor rates!
        F1, F2 = PlanarQuadrotorDynamics.rotor_thrust_model(input.rotor_rates[:2])

        # NOTE(roy) Implement the thrust/torque calculation we discussed in the slides
        u1 = F1 + F2
        u2 = (F1 - F2) * p.arm_length

        # NOTE(roy) this local function is a wrapper around the class state_derivative function
        # all this does is convert the our state dataclass from/into a vector that the scipy
        # integration solver can work with. See dynamics.py for the definition of these
        # from/to state vector functions.
        def state_derivative(t: float, state: np.ndarray) -> np.ndarray:
            return PlanarQuadrotorDynamics.state_derivative(
                QuadrotorState.from_state_vector(state),
                u1,
                u2,
            ).to_state_vector()

        state_vector = self.state.to_state_vector()
        #raise NotImplementedError("Use the scipy function solve_ivp to integrate to the next timestep")
        

        # NOTE(roy) solution = sp.integrate.solve_ivp(....
        solution = sp.integrate.solve_ivp(
            fun=state_derivative,
            t_span=[0, self.dt],
            y0=state_vector,
            method='RK45'
        )

        self.state = QuadrotorState.from_state_vector(solution['y'][:, -1])
        return self.state

    @staticmethod
    def state_derivative(state: QuadrotorState, u1: float, u2: float) -> QuadrotorState:

        # NOTE(roy) compute the relevant values here! Note that we're using only the x- and z dimensions for translation
        # and only the y-axis for rotation for this planar model.
        phi = state.orientation.to_yaw_pitch_roll()[1]
        accel = np.array([-np.sin(phi) / p.mass * u1,
                         0,
                         np.cos(phi) / p.mass * u1 - p.g])
        angular_accel = np.array([0,
                                  u2 / p.inertia,
                                  0])

        # NOTE(roy) this might look a little confusing at first glance, but we're populating DERIVATIVES
        # of the state here! So position -> velocity, velocity -> acceleration, etc...
        # If you can think of a more readable way to do this (e.g. defining a QuadrotorStateDerivative class)
        # feel free to open a pull-request into the upstream repository!
        state_derivative = QuadrotorState(
            position=state.velocity,
            velocity=accel,
            orientation=dRot3(state.orientation, state.angular_velocity),
            angular_velocity=angular_accel,
        )

        return state_derivative

In [6]:
class EmptyTrajectory(TrajectoryBase):
    def eval(self, *args):
        return TrajectoryState(0.0, position=np.zeros(3))  # Trajectory not yet important

class EmptyController(ControllerBase):
    def step(self, *args) -> QuadrotorCommands:
        return QuadrotorCommands(np.array([0, 0, 0, 0]))  # Just zero rotor rates

class HoverController(ControllerBase):
    def step(self, *args) -> QuadrotorCommands:
        return QuadrotorCommands(np.array([598, 598, 0, 0]))  # Hover rate

class ImbalancedController(ControllerBase):
    def step(self, *args) -> QuadrotorCommands:
        return QuadrotorCommands(np.array([600, 500, 0, 0]))  # Imbalanced hover rate

In [7]:
from quadrotor.simulator import SimulatorBase, SimulatorState

# This might be slow to run the first time!
from quadrotor.renderer import animate_k3d

In [8]:
def run_and_render(initial_state: QuadrotorState, t_total: int = 0.5):
    sim = SimulatorBase(
        dt=0.01,
        dynamics=PlanarQuadrotorDynamics(),
        controller=EmptyController(),
        trajectory=EmptyTrajectory(),
        initial_state=initial_state,
        t_total=t_total,
    )
    
    # Run the simulator
    output = sim.simulate()

    # K3D renderer
    plot = animate_k3d(output)
    
    plot.display()
    plot.start_auto_play()

In [9]:
# NOTE(roy) play with this variable and see what happens!
initial_state = QuadrotorState(
    position=np.zeros(3),
    velocity=np.array([0, 0, 0]),
    orientation=Rot3.from_yaw_pitch_roll(0, 0, 0),
    angular_velocity=np.array([0, 0, 0]),
)

run_and_render(initial_state, 1.0)

LinAlgError: 0-dimensional array given. Array must be at least two-dimensional

In [10]:
def run_and_render_hover(initial_state: QuadrotorState, t_total: int = 0.5):
    sim = SimulatorBase(
        dt=0.01,
        dynamics=PlanarQuadrotorDynamics(),
        controller=HoverController(),
        trajectory=EmptyTrajectory(),
        initial_state=initial_state,
        t_total=t_total,
    )
    
    # Run the simulator
    output = sim.simulate()

    # K3D renderer
    plot = animate_k3d(output)
    
    plot.display()
    plot.start_auto_play()

In [11]:
initial_state_hover = QuadrotorState(
    position=np.zeros(3),
    velocity=np.array([0, 0, 0]),
    orientation=Rot3.from_yaw_pitch_roll(0, 0, 0),
    angular_velocity=np.array([0, 0, 0]),
)

run_and_render_hover(initial_state_hover, 1.0)

Output()

In [12]:
def run_and_render_imbalanced(initial_state: QuadrotorState, t_total: int = 0.5):
    sim = SimulatorBase(
        dt=0.01,
        dynamics=PlanarQuadrotorDynamics(),
        controller=ImbalancedController(),
        trajectory=EmptyTrajectory(),
        initial_state=initial_state,
        t_total=t_total,
    )
    
    # Run the simulator
    output = sim.simulate()

    # K3D renderer
    plot = animate_k3d(output)
    
    plot.display()
    plot.start_auto_play()

In [13]:
initial_state_imbalanced = QuadrotorState(
    position=np.zeros(3),
    velocity=np.array([0, 0, 0]),
    orientation=Rot3.from_yaw_pitch_roll(0, 0, 0),
    angular_velocity=np.array([0, 0, 0]),
)

run_and_render_imbalanced(initial_state_imbalanced, 1.0)

Output()

### Does the simulated model make any sense?
If all is well, you should see the quadrotor falling down because of gravity, just as we saw in the slides.

Now try some experiments for yourself and describe whether the behavior is as expected or whether there is anything wrong with your model, ideas to try:
- If you apply an initial upward velocity, what happens?
- If you apply an initial _angular_ velocity, what happens?
- If you spin up both the rotors, what happens (try around 600 rad/s as a start)?
- If you spin one of the rotors faster than the others, what happens?

### Commit your updated notebook to your fork!